In [ ]:
import os
import astropy.units as u
from astropy.coordinates.angles import rotation_matrix
import numpy as np
import matplotlib.pyplot as pl
pl.style.use('apw-notebook')
%matplotlib inline

import gary.coordinates as gc
import gary.dynamics as gd
import gary.integrate as gi
import gary.potential as gp
from gary.units import galactic

In [ ]:
def compute_align_matrix(w):
    x = w[:3].copy()
    v = w[3:].copy()

    # first rotate about z to put on x-z plane
    theta = np.arctan2(x[1], x[0]) * u.radian
    R1 = rotation_matrix(theta, 'z')
    x = np.asarray(R1.dot(x))[0]
    v = np.asarray(R1.dot(v))[0]

    # now rotate about y to put on x axis
    theta = np.arctan2(x[2], x[0]) * u.radian
    R2 = rotation_matrix(-theta, 'y')
    x = np.asarray(R2.dot(x))[0]
    v = np.asarray(R2.dot(v))[0]

    # now align L with z axis
    # theta = np.arccos(L[2] / np.sqrt(np.sum(L**2))) * u.radian
    L = np.cross(x, v)
    theta = np.arctan2(L[2], L[1]) * u.radian
    R3 = rotation_matrix(theta - 90*u.deg, 'x')
    x = np.asarray(R3.dot(x))[0]
    v = np.asarray(R3.dot(v))[0]

    return R3*R2*R1

def align_ensemble(ws):
    R = compute_align_matrix(ws[-1,0])
    new_x = np.array(R.dot(ws[-1,:,:3].T).T)
    new_v = np.array(R.dot(ws[-1,:,3:].T).T)
    new_w = np.vstack((new_x.T, new_v.T)).T
    return new_w

In [ ]:
name = 'strong-chaos'
path = "/Users/adrian/projects/scf_fortran/simulations/runs/{0}/".format(name)

In [ ]:
w = np.genfromtxt(os.path.join(path, "SNAP072"), 
                  usecols=range(1,7), skip_header=1)
cen = np.genfromtxt(os.path.join(path, "SCFCEN"), 
                    usecols=range(2,8), skip_header=1)
w = align_ensemble(np.vstack((cen[-1],w))[None])

## Compute frequencies

In [ ]:
import superfreq as sf

In [ ]:
def compute_all_freqs(t, orbits, hamming_p=1, nintvec=10, force_cartesian=False):
    allfreqs = []
    allamps = []
    for i in range(orbits.shape[1]):
        fs = sf.orbit_to_fs(orbits[:,i])
        sf = SuperFreq(orbits[:,i].t, p=hamming_p)

        try:
            freqs,d,ixs = sf.find_fundamental_frequencies(fs, nintvec=nintvec)
        except:
            allfreqs.append([np.nan,np.nan,np.nan])
            allamps.append([np.nan,np.nan,np.nan])
            continue

        allfreqs.append(freqs.tolist())
        allamps.append(d['|A|'][ixs].tolist())

    allfreqs = np.array(allfreqs)
    allamps = np.array(allamps)

    return allfreqs, allamps

In [ ]:
fig,ax = pl.subplots(1,1,figsize=(6,6))

ax.plot(w[:,0], w[:,1], linestyle='none', alpha=0.4, ms=4, rasterized=True)

ax.set_xlim(-1000,7500)
ax.set_ylim(-4250,4250)
fig.tight_layout()

# _ix = w[:,1] > 0
# ax.plot(w[_ix,0], w[_ix,1], linestyle='none', alpha=0.4, ms=4, rasterized=True)
# ax.plot(w[~_ix,0], w[~_ix,1], linestyle='none', alpha=0.4, ms=4, rasterized=True)
# axes[0].plot(w[~_ix,0] / r_s, 
#              w[~_ix,1] / r_s, 
#              linestyle='none', alpha=0.4, ms=4, rasterized=True)

# axes[0].set_xlim(-1,2)
# axes[0].set_ylim(-1.5,1.5)

# med_ff = np.median(freqs[name][1:], axis=0)[None]
# ff = (freqs[name][1:] - med_ff)/med_ff * 100.
# axes[1].plot(ff[:,0], ff[:,2],
#              linestyle='none', alpha=0.5, ms=4, rasterized=True)

# axes[1].set_xlim(-5,5)
# axes[1].set_ylim(-5,5)
# #     axes[1].set_xlim(-1,1)
# #     axes[1].set_ylim(-1,1)

# axes[0].xaxis.set_visible(False)
# axes[0].yaxis.set_visible(False)

# axes[1].xaxis.set_visible(False)
# axes[1].yaxis.set_visible(False)

# for ax in axes:
#     ax.set_aspect('equal')
# fig.subplots_adjust(wspace=0.1)

In [ ]:
fig,ax = pl.subplots(1,1,figsize=(6,6))

ax.plot(w[:,1], w[:,2], linestyle='none', alpha=0.75, ms=4, rasterized=True)

ax.set_xlim(-3000,3000)
ax.set_ylim(-3000,3000)

fig.tight_layout()

In [ ]:
fig,ax = pl.subplots(1,1,figsize=(6,6))

ax.plot(w[:,0], w[:,1]-80, linestyle='none', alpha=0.75, ms=4, rasterized=True)

ax.set_xlim(6000,7000)
ax.set_ylim(-500,500)

fig.tight_layout()